In [1]:
import pandas as pd

### Load Conversations File

In [37]:
full_conversation_cols = ["Address-A", "Port-A", "Address-B", "Port-B", "Packets", "Bytes", "StreamID", "TotalPackets", 
                          "PerctageFiltered", "Packets_A-to-B(TX_pkts)", "Bytes_A-to-B(TX_bytes)", "Packets_B-to-A(RX_pkts)",
                          "Bytes_B-to-A(RX_bytes)", "Relative_Start", "Duration", "Bits/s A-to-B (TX_bits/s)",
                          "Bits/s B-to-A (RX_bits/s)"]

FLAG_DICT = {"Not set":0, "Set":1}

In [38]:
df_normal_conv = pd.read_csv('Normal_Conversation_list.csv')

In [39]:
df_normal_conv.head()

,Address-A,Port-A,Address-B,Port-B,Packets,Bytes,StreamID,TotalPackets,PerctageFiltered,Packets_A-to-B(TX_pkts),...,Duration,Bits/s A-to-B (TX_bits/s),Bits/s B-to-A (RX_bits/s),tot_kbps,Pktrate,Pktrate nsec,Byte nsec,Flow Duration,Idle Timeout,Hard Timeout
0,96.43.146.176,443,172.16.133.82,61228,1454,97766,0,0,0,67,...,170,246,265.0,7.520462,111.846154,2.005517e-06,0.000135,13,20,100
1,96.43.146.176,443,172.16.133.82,60073,4,380,1,0,0,55,...,253,0,2601.0,0.190000,2.000000,1.970443e-08,0.000002,2,20,100
2,172.16.133.103,63406,216.115.222.200,443,1273,84648,2,0,0,4,...,246,4324,454.0,8.464800,127.300000,7.622754e-06,0.000507,10,20,100
3,172.16.133.43,57700,172.16.139.250,5440,1794,120054,3,0,0,8,...,130,0,12818.0,6.318632,94.421053,4.671875e-06,0.000313,19,20,100
4,172.16.133.53,49292,172.16.139.250,5500,150384,9925344,4,0,0,20,...,242,0,0.0,381.744000,5784.000000,7.595152e-04,0.050128,26,20,100


In [40]:
# df_normal_conv.isna().sum()
df_normal_conv['Bits/s A-to-B (TX_bits/s)'].fillna(0, inplace=True)
df_normal_conv['Bits/s B-to-A (RX_bits/s)'].fillna(0, inplace=True)

### Load Packets File

In [41]:
df_packets = pd.read_csv('Normal_Traffic/normal_traffic_individual_packets.csv')

In [42]:
df_packets['Bytes In Flight'] = df_packets['Bytes In Flight'].fillna(0)

In [43]:
df_packets['Push'] = df_packets['Push'].map(FLAG_DICT)
df_packets['Syn'] = df_packets['Syn'].map(FLAG_DICT)
df_packets['Acknowledgment'] = df_packets['Acknowledgment'].map(FLAG_DICT)
df_packets['Fin'] = df_packets['Fin'].map(FLAG_DICT)
df_packets['Reset'] = df_packets['Reset'].map(FLAG_DICT)

In [44]:
df_packets.head()

,No.,Time,Delta,Source,Destination,Protocol,Stream index,Length,TCP Segment Len,Source Port,Header Length,Bytes In Flight,Push,Syn,Acknowledgment,Fin,Reset,Info
0,4,0.000322,0.000000,96.43.146.176,172.16.133.82,TCP,0,60,0.0,443,20,0.0,0.0,0.0,1.0,0.0,0.0,443 > 61228 [ACK] Seq=1 Ack=1 Win=9659 Len=0
1,7,0.001307,0.000985,96.43.146.176,172.16.133.82,TCP,0,60,0.0,443,20,0.0,0.0,0.0,1.0,0.0,0.0,443 > 61228 [ACK] Seq=1 Ack=1107 Win=10765 L...
2,8,0.005263,0.003956,96.43.146.176,172.16.133.82,TCP,1,60,0.0,443,20,0.0,0.0,0.0,1.0,0.0,0.0,443 > 60073 [ACK] Seq=1 Ack=1 Win=65535 Len=0
3,10,0.006739,0.001476,172.16.133.103,216.115.222.200,TCP,2,66,0.0,63406,32,0.0,0.0,1.0,0.0,0.0,0.0,63406 > 443 [SYN] Seq=0 Win=8192 Len=0 MSS=1...
4,11,0.008991,0.002252,172.16.133.43,172.16.139.250,HTTP,3,265,211.0,57700,20,211.0,1.0,0.0,1.0,0.0,0.0,GET /CSIS/CSISISAPI.dll/?request?aaeb6985-26fc...


### Merge

In [45]:
df_packets['Protocol'].unique()

array(['TCP', 'HTTP', 'TLSv1', 'VNC', 'SSL', 'sFlow', 'SSH', 'TLSv1.1',
       'RDP', 'T.125', 'RTMP', 'SSLv3', 'HTTP/XML', 'DCERPC',
       'IOXIDResolver', 'MPEG TS', 'HTTP/JSON', 'MSNMS', 'SSHv2', 'MP4',
       'XMPP/XML', 'YMSG', 'SSLv2', 'WebSocket', 'FTP', 'IRC', 'OCSP',
       'ICMP', 'NBSS', 'SMB', 'LANMAN', 'RPC_BROWSER', 'FRSTRANS',
       'DRSUAPI', 'EPM', 'POP', 'POP/IMF'], dtype=object)

In [46]:
# df_flag_packet_count = df_packets[['Stream index', 'Syn', 'Acknowledgment', 'Fin', 'Reset']].groupby('Stream index').sum()

In [47]:
# df_conversation = df_normal_conv.merge(df_flag_packet_count, left_on='StreamID', right_index=True)
len(df_packets['Stream index'])

635017

In [48]:
len(df_packets['Stream index'].unique())

23411

In [49]:
len(df_normal_conv)

22312

In [50]:
len(df_packets[df_packets['Protocol'] == 'sFlow'])

1111

In [51]:
23411 - 1111

22300

In [52]:
len(df_packets[df_packets['Protocol'] == 'sFlow']['Stream index'].str.contains(','))

1111

In [53]:
len(df_packets[(df_packets['Protocol'] == 'sFlow') & ~df_packets['Stream index'].str.contains(',')])

11

In [54]:
# df_packets[(df_packets['Protocol'] == 'sFlow') & ~df_packets['Stream index'].str.contains(',')]
len(df_packets[(df_packets['Protocol'] == 'sFlow') & df_packets['Stream index'].str.contains(',')])

1100

In [55]:
23411 - 1100

22311

In [56]:
len(df_packets[df_packets['Stream index'].str.contains(',')])

1100

In [57]:
df_packets_sflow_filter = df_packets[~df_packets['Stream index'].str.contains(',')]

In [58]:
df_flag_packet_count = df_packets_sflow_filter[['Stream index', 'Syn', 'Acknowledgment', 'Fin', 'Reset', 'Push']].groupby('Stream index').sum()

In [59]:
# df_flag_packet_count.head()
df_flag_packet_count.index = df_flag_packet_count.index.astype('float')

In [60]:
df_conversation = df_normal_conv.merge(df_flag_packet_count, left_on='StreamID', right_index=True)

In [61]:
len(df_conversation)

22312

In [62]:
df_packets_sflow_filter[['Stream index', 'Length', 'TCP Segment Len', 'Header Length', 'Bytes In Flight']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 633917 entries, 0 to 635016
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Stream index     633917 non-null  object 
 1   Length           633917 non-null  int64  
 2   TCP Segment Len  633894 non-null  float64
 3   Header Length    633917 non-null  object 
 4   Bytes In Flight  633917 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 29.0+ MB


In [63]:
df_packets_sflow_filter['Stream index'] = df_packets_sflow_filter['Stream index'].astype('int')
df_packets_sflow_filter['Header Length'] = df_packets_sflow_filter['Header Length'].astype('int')

C:\Users\jalaj\AppData\Local\Temp\ipykernel_39888\1995480287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_packets_sflow_filter['Stream index'] = df_packets_sflow_filter['Stream index'].astype('int')
C:\Users\jalaj\AppData\Local\Temp\ipykernel_39888\1995480287.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_packets_sflow_filter['Header Length'] = df_packets_sflow_filter['Header Length'].astype('int')


In [64]:
df_attr_len_mean = df_packets_sflow_filter[['Stream index', 'Length', 'TCP Segment Len', 'Header Length', 'Bytes In Flight']].groupby('Stream index').mean()

In [65]:
df_conversation = df_conversation.merge(df_attr_len_mean, left_on='StreamID', right_index=True)

In [66]:
df_conversation.head()

,Address-A,Port-A,Address-B,Port-B,Packets,Bytes,StreamID,TotalPackets,PerctageFiltered,Packets_A-to-B(TX_pkts),...,Hard Timeout,Syn,Acknowledgment,Fin,Reset,Push,Length,TCP Segment Len,Header Length,Bytes In Flight
0,96.43.146.176,443,172.16.133.82,61228,1454,97766,0,0,0,67,...,100,0.0,113.0,0.0,0.0,33.0,766.761062,710.681416,20.000000,1929.946903
1,96.43.146.176,443,172.16.133.82,60073,4,380,1,0,0,55,...,100,0.0,102.0,0.0,0.0,36.0,742.382353,686.372549,20.000000,2168.274510
2,172.16.133.103,63406,216.115.222.200,443,1273,84648,2,0,0,4,...,100,2.0,5.0,2.0,0.0,0.0,61.000000,0.000000,22.666667,0.000000
3,172.16.133.43,57700,172.16.139.250,5440,1794,120054,3,0,0,8,...,100,0.0,8.0,2.0,0.0,2.0,111.250000,52.750000,20.000000,52.750000
4,172.16.133.53,49292,172.16.139.250,5500,150384,9925344,4,0,0,20,...,100,0.0,20.0,0.0,0.0,10.0,87.000000,30.000000,20.000000,90.000000


In [67]:
df_conversation.isna().sum()

Address-A                    0
Port-A                       0
Address-B                    0
Port-B                       0
Packets                      0
Bytes                        0
StreamID                     0
TotalPackets                 0
PerctageFiltered             0
Packets_A-to-B(TX_pkts)      0
Bytes_A-to-B(TX_bytes)       0
Packets_B-to-A(RX_pkts)      0
Bytes_B-to-A(RX_bytes)       0
Relative_Start               0
Duration                     0
Bits/s A-to-B (TX_bits/s)    0
Bits/s B-to-A (RX_bits/s)    0
tot_kbps                     0
Pktrate                      0
Pktrate nsec                 0
Byte nsec                    0
Flow Duration                0
Idle Timeout                 0
Hard Timeout                 0
Syn                          0
Acknowledgment               0
Fin                          0
Reset                        0
Push                         0
Length                       0
TCP Segment Len              0
Header Length                0
Bytes In

In [68]:
len(df_conversation)

22312

In [69]:
df_conversation.head()

,Address-A,Port-A,Address-B,Port-B,Packets,Bytes,StreamID,TotalPackets,PerctageFiltered,Packets_A-to-B(TX_pkts),...,Hard Timeout,Syn,Acknowledgment,Fin,Reset,Push,Length,TCP Segment Len,Header Length,Bytes In Flight
0,96.43.146.176,443,172.16.133.82,61228,1454,97766,0,0,0,67,...,100,0.0,113.0,0.0,0.0,33.0,766.761062,710.681416,20.000000,1929.946903
1,96.43.146.176,443,172.16.133.82,60073,4,380,1,0,0,55,...,100,0.0,102.0,0.0,0.0,36.0,742.382353,686.372549,20.000000,2168.274510
2,172.16.133.103,63406,216.115.222.200,443,1273,84648,2,0,0,4,...,100,2.0,5.0,2.0,0.0,0.0,61.000000,0.000000,22.666667,0.000000
3,172.16.133.43,57700,172.16.139.250,5440,1794,120054,3,0,0,8,...,100,0.0,8.0,2.0,0.0,2.0,111.250000,52.750000,20.000000,52.750000
4,172.16.133.53,49292,172.16.139.250,5500,150384,9925344,4,0,0,20,...,100,0.0,20.0,0.0,0.0,10.0,87.000000,30.000000,20.000000,90.000000


In [70]:
df_conversation["CATEGORY"] = "Benign Traffic"

In [71]:
df_conversation['flags'] = df_conversation[['Syn', 'Acknowledgment', 'Fin', 'Reset', 'Push']].sum(axis=1)

In [72]:
df_conversation.head()

,Address-A,Port-A,Address-B,Port-B,Packets,Bytes,StreamID,TotalPackets,PerctageFiltered,Packets_A-to-B(TX_pkts),...,Acknowledgment,Fin,Reset,Push,Length,TCP Segment Len,Header Length,Bytes In Flight,CATEGORY,flags
0,96.43.146.176,443,172.16.133.82,61228,1454,97766,0,0,0,67,...,113.0,0.0,0.0,33.0,766.761062,710.681416,20.000000,1929.946903,Benign Traffic,146.0
1,96.43.146.176,443,172.16.133.82,60073,4,380,1,0,0,55,...,102.0,0.0,0.0,36.0,742.382353,686.372549,20.000000,2168.274510,Benign Traffic,138.0
2,172.16.133.103,63406,216.115.222.200,443,1273,84648,2,0,0,4,...,5.0,2.0,0.0,0.0,61.000000,0.000000,22.666667,0.000000,Benign Traffic,9.0
3,172.16.133.43,57700,172.16.139.250,5440,1794,120054,3,0,0,8,...,8.0,2.0,0.0,2.0,111.250000,52.750000,20.000000,52.750000,Benign Traffic,12.0
4,172.16.133.53,49292,172.16.139.250,5500,150384,9925344,4,0,0,20,...,20.0,0.0,0.0,10.0,87.000000,30.000000,20.000000,90.000000,Benign Traffic,30.0


In [73]:
df_conversation.to_csv('Normal_Traffic/normal_traffic_model_ready.csv', index=False)